In [7]:
import math
from itertools import permutations
from typing import List, Tuple, Optional
import subprocess

# ================= НАСТРОЙКИ =====================

MAX_COLS = 10          # макс. число столбцов, которые перебираем
TOP_K_SIMPLE = 50      # сколько лучших по простому скорингу отправляем в LLM
USE_LLM = True         # использовать ли ollama + llama3.2:3b
LLM_MODEL_NAME = "llama3.2:3b"

# ================= НОРМАЛИЗАЦИЯ ==================

def decrypt_denormalize(text: str) -> str:
    """
    Обратная нормализация:
    'прб' -> пробел, 'зпт' -> ',', 'тчк' -> '.'
    """
    return (
        text.replace("тчк", ".")
            .replace("зпт", ",")
            .replace("прб", " ")
    )

# ================= ВОССТАНОВЛЕНИЕ МАТРИЦЫ ========

def decrypt_vertical_with_key(cipher_text: str, key: Tuple[int, ...]) -> Optional[str]:
    """
    Дешифрование вертикальной перестановки при условии, что:
    - текст записывался по строкам в матрицу rows x n_cols,
    - пустые клетки (в конце последней строки) добивались '-',
    - при формировании шифртекста матрица читалась ПО СТОЛБЦАМ
      в порядке, заданном key,
    - символы '-' при чтении НЕ записывались в шифртекст.

    Здесь cipher_text уже БЕЗ пробелов и '-' (как у тебя в GUI).
    """
    cipher_text = cipher_text.lower().replace(" ", "")

    L = len(cipher_text)
    n_cols = len(key)
    if n_cols < 2 or L == 0:
        return None

    # предполагаемое число строк (потолок деления)
    rows = math.ceil(L / n_cols)
    total_cells = rows * n_cols
    empty_cells = total_cells - L  # сколько ячеек были '-'

    # распределение длины по физическим столбцам:
    # считаем, что пустые клетки были в последних столбцах последней строки,
    # значит последние empty_cells столбцов короче на 1.
    col_lens = [rows] * n_cols
    for c in range(n_cols - empty_cells, n_cols):
        if 0 <= c < n_cols:
            col_lens[c] = rows - 1

    # теперь нужно "распилить" cipher_text на столбцы в порядке ЧТЕНИЯ,
    # а затем разложить их на физические позиции.
    cols = [""] * n_cols
    idx = 0
    for pos_in_read_order, col_index_in_matrix in enumerate(key):
        length = col_lens[col_index_in_matrix]
        if length < 0:
            return None
        if idx + length > L:
            # что-то не сходится — такой key/n_cols невозможен
            return None
        cols[col_index_in_matrix] = cipher_text[idx: idx + length]
        idx += length

    # если вдруг не всё из cipher_text израсходовали — тоже странно
    if idx != L:
        # такой вариант считаем некорректным
        return None

    # теперь читаем матрицу по строкам, учитывая,
    # что у последних столбцов может не быть элемента в последней строке
    plain_chars: List[str] = []
    for r in range(rows):
        for c in range(n_cols):
            col_str = cols[c]
            if r < len(col_str):
                plain_chars.append(col_str[r])
            # если r >= len(col_str) — это была та самая '-' ячейка, её не добавляем

    return "".join(plain_chars)

# ================= ПРОСТОЙ СКОРИНГ ===============

COMMON_PATTERNS = [
    " пр ", " не ", " что ", " на ", " но ", " то ", "ст", "ен", " ко", " по "
]

def score_text_simple(text: str) -> int:
    """
    Примитивная оценка осмысленности текста без LLM.
    Больше — лучше.
    """
    t = text.lower()
    score = 0

    for p in COMMON_PATTERNS:
        score += t.count(p) * 5

    # бонусы за пробелы и пунктуацию
    score += t.count(" ") * 2
    score += t.count(".") * 3
    score += t.count(",") * 3

    # лёгкий бонус за длину (очень короткие тексты хуже)
    score += min(len(t), 200) // 10

    return score

# ================= LLaMA через OLLAMA ===========

def score_with_llama_ollama(text: str) -> Optional[float]:
    """
    Оценка осмысленности текста через локальную модель llama3.2:3b
    с использованием 'ollama run'.
    Возвращает число (0..100) или None, если не удалось.
    """
    prompt = (
        "Оцени, насколько этот текст является осмысленным русским предложением "
        "или связным фрагментом текста. "
        "Выведи ТОЛЬКО одно число от 0 до 100.\n\n"
        f"Текст:\n{text}\n"
    )

    try:
        proc = subprocess.run(
            ["ollama", "run", LLM_MODEL_NAME],
            input=prompt.encode("utf-8"),
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            check=False,
        )

        if proc.returncode != 0:
            print("WARN: ollama error:", proc.stderr.decode("utf-8", errors="ignore"))
            return None

        raw = proc.stdout.decode("utf-8", errors="ignore").strip()

        for token in raw.split():
            token = token.replace(",", ".")
            try:
                return float(token)
            except ValueError:
                continue

        return None

    except FileNotFoundError:
        print("WARN: команда 'ollama' не найдена. Работаем без LLM.")
        return None

# ================= БРУТФОРС ======================

def brute_force_vertical_auto(cipher_text: str,
                              max_cols: int = MAX_COLS,
                              top_k_simple: int = TOP_K_SIMPLE,
                              use_llm: bool = USE_LLM):
    """
    Полный брутфорс вертикальной перестановки:
    1) перебираем n_cols от 2 до max_cols (без требования кратности длины),
    2) для каждого n_cols перебираем ВСЕ перестановки столбцов,
    3) считаем простой скоринг, набираем top_k_simple лучших кандидатов,
    4) при наличии LLM — дооцениваем их и выбираем лучший.

    Возвращает:
        (simple_score, n_cols, key, plain_norm, plain_human)
    """
    cipher_text = cipher_text.replace(" ", "").lower()
    L = len(cipher_text)
    if L == 0:
        raise ValueError("Пустой шифртекст.")

    candidates: List[Tuple[float, int, Tuple[int, ...], str, str]] = []

    for n_cols in range(2, min(max_cols, L) + 1):
        print(f"[INFO] Пробуем n_cols = {n_cols} ...")
        for key in permutations(range(n_cols)):
            plain_norm = decrypt_vertical_with_key(cipher_text, key)
            if not plain_norm:
                continue

            plain_human = decrypt_denormalize(plain_norm)
            s = score_text_simple(plain_human)

            if len(candidates) < top_k_simple:
                candidates.append((s, n_cols, key, plain_norm, plain_human))
                candidates.sort(reverse=True, key=lambda x: x[0])
            else:
                if s > candidates[-1][0]:
                    candidates[-1] = (s, n_cols, key, plain_norm, plain_human)
                    candidates.sort(reverse=True, key=lambda x: x[0])

    if not candidates:
        raise ValueError("Не удалось найти ни одного кандидата. Возможно, шифр не вертикальный.")

    print(f"[INFO] Набрано {len(candidates)} лучших кандидатов по простому скорингу.")

    # если LLM отключена — берём лучшего по простому скорингу
    if not use_llm:
        return candidates[0]

    # дооцениваем через LLaMA
    best_llm_score = -1e9
    best_tuple = None

    for s_simple, n_cols, key, plain_norm, plain_human in candidates:
        llm_score = score_with_llama_ollama(plain_human)
        if llm_score is None:
            llm_score = float(s_simple)

        print(f"[LLM] n_cols={n_cols}, key={key}, simple={s_simple}, llm={llm_score}")
        if llm_score > best_llm_score:
            best_llm_score = llm_score
            best_tuple = (s_simple, n_cols, key, plain_norm, plain_human)

    return best_tuple

# ================= MAIN ==========================

def main():
    print("=== Взлом вертикальной перестановки ===")
    cipher = input("Введите шифртекст (можно с пробелами групп): ").strip()

    simple_score, n_cols, key, plain_norm, plain_human = brute_force_vertical_auto(cipher)

    print("\n=== РЕЗУЛЬТАТ ===")
    print(f"Число столбцов: {n_cols}")
    print(f"Ключ (перестановка): {key}")
    print(f"Простой скоринг: {simple_score}")
    print("\nНормализованный текст (с 'прб', 'зпт', 'тчк'):")
    print(plain_norm)
    print("\nРасшифрованный текст:")
    print(plain_human)


if __name__ == "__main__":
    main()


=== Взлом вертикальной перестановки ===
[INFO] Пробуем n_cols = 2 ...
[INFO] Пробуем n_cols = 3 ...
[INFO] Пробуем n_cols = 4 ...
[INFO] Пробуем n_cols = 5 ...
[INFO] Пробуем n_cols = 6 ...
[INFO] Пробуем n_cols = 7 ...
[INFO] Пробуем n_cols = 8 ...
[INFO] Пробуем n_cols = 9 ...
[INFO] Пробуем n_cols = 10 ...
[INFO] Набрано 50 лучших кандидатов по простому скорингу.
[LLM] n_cols=5, key=(1, 2, 3, 0, 4), simple=31, llm=80.0
[LLM] n_cols=5, key=(2, 3, 4, 1, 0), simple=28, llm=28.0
[LLM] n_cols=10, key=(4, 6, 5, 7, 1, 8, 3, 0, 2, 9), simple=28, llm=55.0
[LLM] n_cols=10, key=(4, 7, 6, 3, 1, 8, 5, 0, 2, 9), simple=28, llm=65.0
[LLM] n_cols=10, key=(9, 6, 1, 7, 3, 8, 0, 5, 4, 2), simple=28, llm=0.0
[LLM] n_cols=10, key=(9, 6, 2, 7, 4, 8, 0, 5, 1, 3), simple=28, llm=40.0
[LLM] n_cols=10, key=(6, 0, 9, 1, 8, 2, 7, 4, 3, 5), simple=27, llm=0.0
[LLM] n_cols=10, key=(7, 0, 9, 1, 6, 2, 8, 4, 3, 5), simple=27, llm=45.0
[LLM] n_cols=10, key=(7, 5, 9, 6, 1, 3, 8, 4, 2, 0), simple=27, llm=40.0
[LLM] n_

In [15]:
from itertools import permutations
import time
score = 0
for n in range(6, 14, 2):
    start = time.time()
    for p in permutations(list(range(n)), n):
        score += 1
    end = time.time()
    print(n, "конец", score, end - start)

6 конец 720 0.0
8 конец 41040 0.0029981136322021484
10 конец 3669840 0.2803688049316406
12 конец 482671440 31.37127375602722
